## 사용자 관리 기능을 추가 합시다.

사용자 관리 기능을 추가하기 위해 먼저 필요한 API 엔드포인트들을 구현하겠습니다. 사용자 등록, 로그인, 조회 등의 기본적인 기능을 추가하겠습니다.

먼저 파일을 수정하겠습니다:


```javascript:server/index.js
require('dotenv').config();

const express = require('express');
const cors = require('cors');
const { Pool } = require('pg');
const bcrypt = require('bcrypt');
const jwt = require('jsonwebtoken');
const app = express();
const port = 5001;

// CORS 설정
app.use(cors());
app.use(express.json());

// PostgreSQL 연결 설정
const pool = new Pool({
  user: process.env.DB_USER,
  host: process.env.DB_HOST,
  database: process.env.DB_NAME,
  password: process.env.DB_PASSWORD,
  port: process.env.DB_PORT,
});

// JWT 시크릿 키
const JWT_SECRET = process.env.JWT_SECRET || 'your-secret-key';

// 사용자 등록 API
app.post('/api/register', async (req, res) => {
  try {
    const { username, email, password } = req.body;
    
    // 비밀번호 해시화
    const hashedPassword = await bcrypt.hash(password, 10);
    
    // 사용자 등록
    const result = await pool.query(
      'INSERT INTO users (username, email, password) VALUES ($1, $2, $3) RETURNING id, username, email',
      [username, email, hashedPassword]
    );
    
    res.status(201).json(result.rows[0]);
  } catch (err) {
    console.error(err);
    res.status(500).json({ error: '회원가입 중 오류가 발생했습니다.' });
  }
});

// 로그인 API
app.post('/api/login', async (req, res) => {
  try {
    const { email, password } = req.body;
    
    // 사용자 조회
    const result = await pool.query('SELECT * FROM users WHERE email = $1', [email]);
    const user = result.rows[0];
    
    if (!user) {
      return res.status(401).json({ error: '이메일 또는 비밀번호가 잘못되었습니다.' });
    }
    
    // 비밀번호 확인
    const validPassword = await bcrypt.compare(password, user.password);
    if (!validPassword) {
      return res.status(401).json({ error: '이메일 또는 비밀번호가 잘못되었습니다.' });
    }
    
    // JWT 토큰 생성
    const token = jwt.sign(
      { userId: user.id, email: user.email },
      JWT_SECRET,
      { expiresIn: '24h' }
    );
    
    res.json({ token, user: { id: user.id, username: user.username, email: user.email } });
  } catch (err) {
    console.error(err);
    res.status(500).json({ error: '로그인 중 오류가 발생했습니다.' });
  }
});

// 사용자 정보 조회 API
app.get('/api/user/:id', async (req, res) => {
  try {
    const result = await pool.query(
      'SELECT id, username, email, created_at FROM users WHERE id = $1',
      [req.params.id]
    );
    
    if (result.rows.length === 0) {
      return res.status(404).json({ error: '사용자를 찾을 수 없습니다.' });
    }
    
    res.json(result.rows[0]);
  } catch (err) {
    console.error(err);
    res.status(500).json({ error: '사용자 정보 조회 중 오류가 발생했습니다.' });
  }
});

// 기본 API 엔드포인트
app.get('/api/itemlist', async (req, res) => {
  try {
    const result = await pool.query('SELECT * FROM api_item');
    res.json(result.rows);
  } catch (err) {
    console.error(err);
    res.status(500).send('Server Error');
  }
});

app.listen(port, () => {
  console.log(`Server running on http://localhost:${port}`);
});
```
이제 사용자 관리를 위한 데이터베이스 테이블을 생성해야 합니다. PostgreSQL에서 users 테이블을 생성하는 SQL 스크립트를 작성하겠습니다:


```ms sql:server/create_users_table.sql
CREATE TABLE IF NOT EXISTS users (
    id SERIAL PRIMARY KEY,
    username VARCHAR(50) NOT NULL,
    email VARCHAR(100) UNIQUE NOT NULL,
    password VARCHAR(255) NOT NULL,
    created_at TIMESTAMP WITH TIME ZONE DEFAULT CURRENT_TIMESTAMP
);
```
이제 필요한 패키지들을 설치해야 합니다. package.json에 의존성을 추가하겠습니다:


```json:server/package.json
{
  "name": "server",
  "version": "1.0.0",
  "description": "",
  "main": "index.js",
  "scripts": {
    "start": "node index.js",
    "dev": "nodemon index.js"
  },
  "keywords": [],
  "author": "",
  "license": "ISC",
  "dependencies": {
    "bcrypt": "^5.1.1",
    "cors": "^2.8.5",
    "dotenv": "^16.4.1",
    "express": "^4.18.2",
    "jsonwebtoken": "^9.0.2",
    "pg": "^8.11.3"
  },
  "devDependencies": {
    "nodemon": "^3.0.3"
  }
}
```
이제 사용자 관리 기능이 추가되었습니다. 구현된 기능은 다음과 같습니다:

1. 사용자 등록 (`POST /api/register`)
   - 사용자명, 이메일, 비밀번호를 받아 새로운 사용자를 등록
   - 비밀번호는 bcrypt로 해시화하여 저장

2. 로그인 (`POST /api/login`)
   - 이메일과 비밀번호로 로그인
   - 성공 시 JWT 토큰과 사용자 정보 반환

3. 사용자 정보 조회 (`GET /api/user/:id`)
   - 특정 사용자의 정보를 조회 (비밀번호 제외)

사용하기 위해서는 다음 단계를 수행해야 합니다:

1. 새로운 패키지 설치:
```bash
npm install
```

2. PostgreSQL에서 users 테이블 생성:
```bash
psql -U your_username -d your_database -f create_users_table.sql
```

3. `.env` 파일에 JWT 시크릿 키 추가:
```
JWT_SECRET=your-secure-secret-key
```

API 테스트는 다음과 같이 할 수 있습니다:

1. 사용자 등록:
```bash
curl -X POST http://localhost:5001/api/register \
  -H "Content-Type: application/json" \
  -d '{"username":"testuser", "email":"test@example.com", "password":"password123"}'
```

2. 로그인:
```bash
curl -X POST http://localhost:5001/api/login \
  -H "Content-Type: application/json" \
  -d '{"email":"test@example.com", "password":"password123"}'
```

3. 사용자 정보 조회:
```bash
curl http://localhost:5001/api/user/1
```

추가로 필요한 기능이나 수정사항이 있다면 말씀해 주세요!
